## Promoting Tourism in San Francisco
<p>San Francisco has been home to many famous films, including the action classic “Bullitt” and the recent science-fiction epic “Rise of the Planet of the Apes”. To celebrate the cinematic history of the city, the tourism board has asked you to perform some analyses.</p>
<p>Their idea is to promote the 10 most popular filming locations in San Franciso. The board plans to create an attraction at each of the 10 locations based on the biggest film (by worldwide income) shot there.</p>
<p>At your disposal are two datasets. One contains every location and film shot in San Franciso. The other dataset contains movie details drawn from the Internet Movie Database (IMDB). </p>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6;">
    <div style="font-size:16px"><b>datasets/locations.csv - Filming locations of movies shot in San Francisco since 1924</b>
    </div>
    <div> Source: <a href="https://data.sfgov.org/Culture-and-Recreation/Film-Locations-in-San-Francisco/yitu-d5am">Film Locations in San Francisco</a></div>

<ul>
    <li><b>Title: </b>Title of the movie. Note that some films may share the same title, and are only differentiated by year of release.</li>
    <li><b>Release Year: </b>Year of release in cinemas.</li>
    <li><b>Locations: </b>Name of location in San Francisco where a scene was shot for the movie.</li>
    <li><b>Production Company: </b>Company that produced the film.</li>
    <li><b>Distributor: </b>Company that distributed the film.</li>
</ul>
    </div>
<div style="background-color: #efebe4; color: #05192d; text-align:left; vertical-align: middle; padding: 15px 25px 15px 25px; line-height: 1.6; margin-top: 17px;">
    <div style="font-size:16px"><b>datasets/imdb_movies.csv - Data on over 85,000 movies up to 2020</b>
    </div>
    <div>Source: <a href="https://www.kaggle.com/stefanoleone992/imdb-extensive-dataset">Kaggle (IMDb movies extensive dataset)</a></div>
<ul>
    <li><b>imdb_title_id: </b>Unique film id.</li>
    <li><b>title: </b>Title of the film. Note that some films may share the same title, and are only differentiated by year of release.</li>
    <li><b>year: </b>The year of release.</li> 
    <li><b>genre: </b>The genres of the film. The primary genre of the film is the first genre listed.</li>
    <li><b>duration: </b>The duration of the film in minutes.</li>
    <li><b>director: </b>The name of the director.</li>
    <li><b>actors: </b>The leading actors of the film.</li>
    <li><b>avg_vote: </b>Average review given to the film.</li>
    <li><b>worldwide_gross_income: </b>Total income for the film worldwide in US dollars.</li>
</ul>
    </div>

### Import neccesary libaries

In [4]:
# Use this cell to begin your analysis, and add as many as you would like!
import pandas as pd
import numpy as np

### Reading the files

In [5]:
loc = pd.read_csv('datasets/locations.csv')
imdb = pd.read_csv('datasets/imdb_movies.csv')

### EDA (Exploratory Data Analysis)

In [6]:
loc.dtypes

Title                 object
Release Year           int64
Locations             object
Production Company    object
Distributor           object
dtype: object

In [7]:
# Looking at the info, maybe we got a issue with the worldwide_gross_income column bc of NaN and type (object)
imdb.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85854 entries, 0 to 85853
Data columns (total 9 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   imdb_title_id           85854 non-null  object 
 1   title                   85854 non-null  object 
 2   year                    85854 non-null  int64  
 3   genre                   85854 non-null  object 
 4   duration                85854 non-null  int64  
 5   director                85767 non-null  object 
 6   actors                  85785 non-null  object 
 7   avg_vote                85854 non-null  float64
 8   worldwide_gross_income  31016 non-null  object 
dtypes: float64(1), int64(2), object(6)
memory usage: 5.9+ MB


In [8]:
loc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1743 entries, 0 to 1742
Data columns (total 5 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Title               1743 non-null   object
 1   Release Year        1743 non-null   int64 
 2   Locations           1689 non-null   object
 3   Production Company  1741 non-null   object
 4   Distributor         1642 non-null   object
dtypes: int64(1), object(4)
memory usage: 68.2+ KB


### Cleaning and preparing for analysis

##### Drop rows of NaN

In [9]:
imdb.isnull().sum()

imdb_title_id                 0
title                         0
year                          0
genre                         0
duration                      0
director                     87
actors                       69
avg_vote                      0
worldwide_gross_income    54838
dtype: int64

In [10]:
imdb.dropna(how='any', subset=['worldwide_gross_income'],inplace = True)
imdb.isnull().sum()

imdb_title_id              0
title                      0
year                       0
genre                      0
duration                   0
director                  20
actors                    24
avg_vote                   0
worldwide_gross_income     0
dtype: int64

In [11]:
loc.isnull().sum()

Title                   0
Release Year            0
Locations              54
Production Company      2
Distributor           101
dtype: int64

In [12]:
loc.dropna(how='any', subset=['Locations'],inplace = True)
loc.isnull().sum()

Title                   0
Release Year            0
Locations               0
Production Company      2
Distributor           100
dtype: int64

##### Converting income to dollars

In [13]:
imdb.worldwide_gross_income.value_counts().head(20)

$ 8144           15
$ 46808          10
$ 509             9
$ 97182           6
$ 2874            5
$ 14000000        5
$ 2248            4
INR 220000000     4
$ 1500000         4
$ 11000000        4
$ 12000000        4
INR 320000000     3
$ 10000000        3
$ 5000            3
$ 852             3
$ 3000000         3
$ 12751           3
$ 15035           3
$ 2735            3
$ 8462            3
Name: worldwide_gross_income, dtype: int64

In [14]:
# It seems there are INR which is Indian Rupees where 1 INR = 0.013 dollars
# Let's see if there are others currencies
imdb['currency'] = imdb['worldwide_gross_income'].map(lambda x: x.split(' ')[0])
imdb['income'] = imdb['worldwide_gross_income'].map(lambda x: x.split(' ')[1])
imdb.head()

,imdb_title_id,title,year,genre,duration,director,actors,avg_vote,worldwide_gross_income,currency,income
79,tt0007183,Pikovaya dama,1916,"Drama, Fantasy, Horror",63,Yakov Protazanov,"Tamara Duvan, Ivan Mozzhukhin, Vera Orlova, Ni...",7.0,$ 144968,$,144968
165,tt0010323,Das Cabinet des Dr. Caligari,1920,"Fantasy, Horror, Mystery",76,Robert Wiene,"Werner Krauss, Conrad Veidt, Friedrich Feher, ...",8.1,$ 8811,$,8811
210,tt0011440,Markens grøde,1921,Drama,107,Gunnar Sommerfeldt,"Amund Rydland, Karen Poulsen, Ragna Wettergree...",6.6,$ 4272,$,4272
222,tt0011741,Suds,1920,"Comedy, Drama, Romance",75,John Francis Dillon,"Mary Pickford, Albert Austin, Harold Goodwin, ...",6.3,$ 772155,$,772155
245,tt0012190,The Four Horsemen of the Apocalypse,1921,"Drama, Romance, War",150,Rex Ingram,"Pomeroy Cannon, Josef Swickard, Bridgetta Clar...",7.2,$ 9183673,$,9183673


In [15]:
# More of them... PKR (Pakistanee rupiee 0,0056 $),INR (Indian rupiee 0,013 $) GBP (Pounds 1,32 $) and NPR (Nepalee Rupiee 0,0082 $)
imdb.currency.value_counts()

$      30955
INR       58
PKR        1
GBP        1
NPR        1
Name: currency, dtype: int64

In [16]:
currencies = ['PKR','GBP','NPR']
imdb[imdb['currency'].isin(currencies)]

,imdb_title_id,title,year,genre,duration,director,actors,avg_vote,worldwide_gross_income,currency,income
3525,tt0032359,Convoy,1940,"Drama, War",90,Pen Tennyson,"Clive Brook, John Clements, Edward Chapman, Ju...",6.2,GBP 50000,GBP,50000
72026,tt3638644,Seto Surya,2016,"Adventure, Drama",89,Deepak Rauniyar,"Dayahang Rai, Asha Magrati, Rabindra Singh Ban...",7.1,NPR 20000000,NPR,20000000
75255,tt4635548,Wrong No.,2015,"Comedy, Romance",133,Yasir Nawaz,"Javed Sheikh, Danish Taimoor, Janita Asma, Nay...",6.7,PKR 157500000,PKR,157500000


In [17]:
# Converting column income to dolars (numeric)
imdb.income = pd.to_numeric(imdb.income)
imdb.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 31016 entries, 79 to 85853
Data columns (total 11 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   imdb_title_id           31016 non-null  object 
 1   title                   31016 non-null  object 
 2   year                    31016 non-null  int64  
 3   genre                   31016 non-null  object 
 4   duration                31016 non-null  int64  
 5   director                30996 non-null  object 
 6   actors                  30992 non-null  object 
 7   avg_vote                31016 non-null  float64
 8   worldwide_gross_income  31016 non-null  object 
 9   currency                31016 non-null  object 
 10  income                  31016 non-null  int64  
dtypes: float64(1), int64(3), object(7)
memory usage: 2.8+ MB


In [18]:
def to_dollar(col):
    currency = col[0]
    income = col[1]
    
    if currency == '$':
        return income
    elif currency == 'INR':
        return income * 0.013
    elif currency == 'GBP':
        return income * 1.32
    elif currency == 'NPR':
        return income * 0.0056
    elif currency == 'PKR':
        return income * 0.0082
    else:
        return income

In [19]:
imdb['income_dollar'] = imdb[['currency','income']].apply(to_dollar,axis=1)
imdb.drop(columns=['worldwide_gross_income','currency','income','imdb_title_id','duration','director','actors'], inplace = True)

# In order to have an accurate index we should reset it
imdb.reset_index(inplace=True,drop=True)
imdb.tail(5)

,title,year,genre,avg_vote,income_dollar
31011,Ottam,2019,Drama,7.4,4791.0
31012,Le lion,2020,Comedy,5.3,3507171.0
31013,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",7.7,7299062.0
31014,Sokagin Çocuklari,2019,"Drama, Family",6.4,2833.0
31015,La vida sense la Sara Amat,2019,Drama,6.7,59794.0


##### Limiting the results to movies by genre and avg > 6

In [20]:
# First: We need to split the the string in the column genre to get the first string which is the first genre.
imdb["genre"] = imdb["genre"].map(lambda x: x.split(', ')[0])
imdb.tail(5)

,title,year,genre,avg_vote,income_dollar
31011,Ottam,2019,Drama,7.4,4791.0
31012,Le lion,2020,Comedy,5.3,3507171.0
31013,De Beentjes van Sint-Hildegard,2020,Comedy,7.7,7299062.0
31014,Sokagin Çocuklari,2019,Drama,6.4,2833.0
31015,La vida sense la Sara Amat,2019,Drama,6.7,59794.0


In [21]:
imdb.genre.value_counts()

Comedy         9729
Drama          8971
Action         4579
Crime          1829
Animation      1322
Biography      1171
Horror         1159
Adventure      1145
Thriller        277
Romance         218
Family          167
Fantasy         141
Mystery         133
Musical          54
Sci-Fi           40
History          23
Western          18
Music            18
War               9
Sport             7
Film-Noir         5
Documentary       1
Name: genre, dtype: int64

In [22]:
# Second: films with avg_vote > 6 & genre = Action, Drama, Biography.
filter_list = ['Biography', 'Drama', 'Action']
imdb = imdb.loc[(imdb['avg_vote'] > 6.0) & (imdb['genre'].isin(filter_list))]
imdb.reset_index(inplace=True,drop=True)
imdb.tail(5)

,title,year,genre,avg_vote,income_dollar
9547,VFW,2019,Action,6.1,23101.0
9548,Kaithi,2019,Action,8.5,524061.0
9549,Ottam,2019,Drama,7.4,4791.0
9550,Sokagin Çocuklari,2019,Drama,6.4,2833.0
9551,La vida sense la Sara Amat,2019,Drama,6.7,59794.0


##### Concatenate title and year to compare with location dataset

In [23]:
# some films may share the same title, and are only differentiated by year of release. Need to concat and look up by it.
cols = ['title','year']
imdb['concat'] = imdb[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
imdb.tail(5)

,title,year,genre,avg_vote,income_dollar,concat
9547,VFW,2019,Action,6.1,23101.0,VFW_2019
9548,Kaithi,2019,Action,8.5,524061.0,Kaithi_2019
9549,Ottam,2019,Drama,7.4,4791.0,Ottam_2019
9550,Sokagin Çocuklari,2019,Drama,6.4,2833.0,Sokagin Çocuklari_2019
9551,La vida sense la Sara Amat,2019,Drama,6.7,59794.0,La vida sense la Sara Amat_2019


In [24]:
cols = ['Title','Release Year']
loc['concat'] = loc[cols].apply(lambda row: '_'.join(row.values.astype(str)), axis=1)
loc.tail(5)

,Title,Release Year,Locations,Production Company,Distributor,concat
1738,Always Be My Maybe,2019,Pier 7,"Isla Productions, LLC",Netflix,Always Be My Maybe_2019
1739,The Last Black Man in San Francisco,2019,Castro and Market,"LBMISF, LLC",A47,The Last Black Man in San Francisco_2019
1740,Tales of the City,2019,Castro District,Universal Television LLC,Netflix,Tales of the City_2019
1741,Tales of the City,2019,Mission District,Universal Television LLC,Netflix,Tales of the City_2019
1742,Bullitt,1968,Coit Tower,Warner Brothers / Seven Arts\nSeven Arts,Warner Brothers,Bullitt_1968


In [25]:
# Dropping columns no needed from loc dataset
drop_col = ['Production Company','Distributor']
loc.drop(columns=drop_col, inplace = True)
loc.tail(5)

,Title,Release Year,Locations,concat
1738,Always Be My Maybe,2019,Pier 7,Always Be My Maybe_2019
1739,The Last Black Man in San Francisco,2019,Castro and Market,The Last Black Man in San Francisco_2019
1740,Tales of the City,2019,Castro District,Tales of the City_2019
1741,Tales of the City,2019,Mission District,Tales of the City_2019
1742,Bullitt,1968,Coit Tower,Bullitt_1968


###### Extracting the popular locations

In [26]:
loc['Locations'].value_counts().head(10)

Golden Gate Bridge                                           27
City Hall                                                    22
Fairmont Hotel (950 Mason Street, Nob Hill)                  21
Treasure Island                                              14
Coit Tower                                                   12
Palace of Fine Arts (3301 Lyon Street)                       11
Chinatown                                                    10
Bay Bridge                                                    9
Grace Cathedral Episcopal Church (1100 California Street)     8
Hall of Justice (850 Bryant Street)                           7
Name: Locations, dtype: int64

In [27]:
pop = loc.groupby(['Locations']).size().reset_index(name='Count')
pop_locations = pop.sort_values(by='Count',ascending=False)
pop_locations.head(10)

,Locations,Count
668,Golden Gate Bridge,27
497,City Hall,22
600,"Fairmont Hotel (950 Mason Street, Nob Hill)",21
1264,Treasure Island,14
519,Coit Tower,12
1018,Palace of Fine Arts (3301 Lyon Street),11
486,Chinatown,10
358,Bay Bridge,9
673,Grace Cathedral Episcopal Church (1100 Califor...,8
709,Hall of Justice (850 Bryant Street),7


In [28]:
loc_count = loc.merge(pop_locations,how='left', on='Locations')
loc_count.sort_values(by='Count',ascending=False)

,Title,Release Year,Locations,concat,Count
488,Looking,2014,Golden Gate Bridge,Looking_2014,27
1542,Star Trek IV: The Voyage Home,1986,Golden Gate Bridge,Star Trek IV: The Voyage Home_1986,27
833,The Core,2003,Golden Gate Bridge,The Core_2003,27
1452,It Came From Beneath the Sea,1955,Golden Gate Bridge,It Came From Beneath the Sea_1955,27
1455,Mother,1996,Golden Gate Bridge,Mother_1996,27
...,...,...,...,...,...
104,Basic Instinct,1992,2104 Broadway,Basic Instinct_1992,1
707,"Smile Again, Jenny Lee",2015,Hyatt Hotel (555 North Point St.),"Smile Again, Jenny Lee_2015",1
706,"Smile Again, Jenny Lee",2015,475 Diamond St.,"Smile Again, Jenny Lee_2015",1
705,"Smile Again, Jenny Lee",2015,Manor West (750 Harrison St.),"Smile Again, Jenny Lee_2015",1


##### Merging datasets and sorting by incomes

In [29]:
# Merging in a single df by the concatenated row
# merged = pd.merge(imdb, loc[['concat','Locations']], on='concat')
merged = imdb.merge(loc_count, how='inner', on='concat').sort_values(by=['Count','income_dollar'], ascending = False)
merged.reset_index(inplace=True,drop=True)
merged.head()

,title,year,genre,avg_vote,income_dollar,concat,Title,Release Year,Locations,Count
0,Superman,1978,Action,7.3,300451667.0,Superman_1978,Superman,1978,Golden Gate Bridge,27
1,Star Trek VI: The Undiscovered Country,1991,Action,7.2,96888996.0,Star Trek VI: The Undiscovered Country_1991,Star Trek VI: The Undiscovered Country,1991,Golden Gate Bridge,27
2,Milk,2008,Biography,7.5,54589558.0,Milk_2008,Milk,2008,Golden Gate Bridge,27
3,A View to a Kill,1985,Action,6.4,50327960.0,A View to a Kill_1985,A View to a Kill,1985,Golden Gate Bridge,27
4,Jagged Edge,1985,Drama,6.5,40491165.0,Jagged Edge_1985,Jagged Edge,1985,Golden Gate Bridge,27


In [30]:
# Auxiliar dataframe to get an array with the positions we are looking for
aux_df = merged[['Locations','income_dollar']]
aux_df = aux_df.groupby(['Locations'], sort=False)['income_dollar'].idxmax()

final_df = merged.iloc[aux_df]
final_df.head()

,title,year,genre,avg_vote,income_dollar,concat,Title,Release Year,Locations,Count
0,Superman,1978,Action,7.3,300451667.0,Superman_1978,Superman,1978,Golden Gate Bridge,27
8,Dawn of the Planet of the Apes,2014,Action,7.6,710644566.0,Dawn of the Planet of the Apes_2014,Dawn of the Planet of the Apes,2014,City Hall,22
17,The Rock,1996,Action,7.4,335062621.0,The Rock_1996,The Rock,1996,"Fairmont Hotel (950 Mason Street, Nob Hill)",21
22,Patch Adams,1998,Biography,6.8,202292902.0,Patch Adams_1998,Patch Adams,1998,Treasure Island,14
28,San Andreas,2015,Action,6.1,473990832.0,San Andreas_2015,San Andreas,2015,Coit Tower,12


In [31]:
final_df.reset_index(inplace=True, drop=True)
final_df.head()

,title,year,genre,avg_vote,income_dollar,concat,Title,Release Year,Locations,Count
0,Superman,1978,Action,7.3,300451667.0,Superman_1978,Superman,1978,Golden Gate Bridge,27
1,Dawn of the Planet of the Apes,2014,Action,7.6,710644566.0,Dawn of the Planet of the Apes_2014,Dawn of the Planet of the Apes,2014,City Hall,22
2,The Rock,1996,Action,7.4,335062621.0,The Rock_1996,The Rock,1996,"Fairmont Hotel (950 Mason Street, Nob Hill)",21
3,Patch Adams,1998,Biography,6.8,202292902.0,Patch Adams_1998,Patch Adams,1998,Treasure Island,14
4,San Andreas,2015,Action,6.1,473990832.0,San Andreas_2015,San Andreas,2015,Coit Tower,12


In [32]:
final_df.rename(columns={"year": "Year","Locations":"Location"},inplace=True)
final_df.head()

C:\Users\Luis Andres\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\LocalCache\local-packages\Python39\site-packages\pandas\core\frame.py:4441: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,title,Year,genre,avg_vote,income_dollar,concat,Title,Release Year,Location,Count
0,Superman,1978,Action,7.3,300451667.0,Superman_1978,Superman,1978,Golden Gate Bridge,27
1,Dawn of the Planet of the Apes,2014,Action,7.6,710644566.0,Dawn of the Planet of the Apes_2014,Dawn of the Planet of the Apes,2014,City Hall,22
2,The Rock,1996,Action,7.4,335062621.0,The Rock_1996,The Rock,1996,"Fairmont Hotel (950 Mason Street, Nob Hill)",21
3,Patch Adams,1998,Biography,6.8,202292902.0,Patch Adams_1998,Patch Adams,1998,Treasure Island,14
4,San Andreas,2015,Action,6.1,473990832.0,San Andreas_2015,San Andreas,2015,Coit Tower,12


###### Dropping last columns leaving just the 3 columns requested

In [33]:
sf_hits = final_df[['Location','Title','Year']].loc[0:9]
sf_hits.head(20)

,Location,Title,Year
0,Golden Gate Bridge,Superman,1978
1,City Hall,Dawn of the Planet of the Apes,2014
2,"Fairmont Hotel (950 Mason Street, Nob Hill)",The Rock,1996
3,Treasure Island,Patch Adams,1998
4,Coit Tower,San Andreas,2015
5,Palace of Fine Arts (3301 Lyon Street),Forrest Gump,1994
6,Chinatown,Basic Instinct,1992
7,Bay Bridge,The Game,1997
8,Grace Cathedral Episcopal Church (1100 Califor...,The Towering Inferno,1974
9,Hall of Justice (850 Bryant Street),Basic Instinct,1992
